In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import os
import re
import glob
import pandas as pd
from natsort import natsorted
import unicodedata

### Variabili Globali

In [25]:
root_folder = 'drive/MyDrive/BERT/'
data_folder_in = 'data/Dante/'
data_folder_out = 'data/'

# PATH
DATA_PATH_IN = os.path.abspath(os.path.join(root_folder, data_folder_in))
DATA_PATH_OUT = os.path.abspath(os.path.join(root_folder, data_folder_out))

# File utilizzati per l'addestramento
data_filename = ['fiore_it.csv',
                 'fiore_it_en.csv',
                 'fiore_it_de.csv',
                 'fiore_it_fr.csv',
                 'fiore_it_es.csv',
                 'inferno_v2.csv',
                 'inferno_it_fr.csv',
                 'inferno_it_es.csv',
                 'inferno_it_de.csv',
                 'inferno_en.csv',
                 'inferno_gpt_en.csv',
                 'inferno_gpt_it_fr.csv',
                 'inferno_gpt_it_es.csv',
                 'inferno_gpt_it_de.csv',
                 'purgatorio_v2.csv',
                 'purgatorio_it_fr.csv',
                 'purgatorio_it_es.csv',
                 'purgatorio_it_de.csv',
                 'purgatorio_en.csv',
                 'purgatorio_gpt_en.csv',
                 'purgatorio_gpt_it_fr.csv',
                 'purgatorio_gpt_it_es.csv',
                 'purgatorio_gpt_it_de.csv',
                 'paradiso_v2.csv',
                 'paradiso_it_fr.csv',
                 'paradiso_it_es.csv',
                 'paradiso_it_de.csv',
                 'paradiso_en.csv',
                 'paradiso_gpt_en.csv',
                 'paradiso_gpt_it_fr.csv',
                 'paradiso_gpt_it_es.csv',
                 'paradiso_gpt_it_de.csv']

# Colonne
ORIGINAL_COLUMN = 'Original'
TRANSLATE_COLUMN = 'Translate'
TRANSLATE_COLUMN_GPT = 'Translate_GPT_1'

file_out = 'dataset_dantesco.csv'

In [26]:
def verifica_correttezza(df):
  original = df['Original']
  translate = df['Translate']

  print(f'Esempi nel Dataset                 : {len(df)}')
  print(f'Frase più corta in Original        : {min(original, key = len)}')
  print(f'Frase più corta in Translate       : {min(translate, key = len)}')
  print(f'Frase più lunga in Original        : {max(original, key = len)}')
  print(f'Frase più lunga in Translate       : {max(translate, key = len)}')
  print('---------------------------------------------------------------------------------------')

In [27]:
def union_df_train():
  df = pd.DataFrame(columns=[ORIGINAL_COLUMN, TRANSLATE_COLUMN])
  
  for filename in data_filename:
    filename_path = os.path.abspath(os.path.join(DATA_PATH_IN, filename))

    if "fiore" in filename:
      df_train = pd.read_csv(
        filename_path,
        usecols=[ORIGINAL_COLUMN, TRANSLATE_COLUMN, TRANSLATE_COLUMN_GPT],
        dtype={ORIGINAL_COLUMN: str, TRANSLATE_COLUMN: str, TRANSLATE_COLUMN_GPT: str}
      )

      df_train = pd.concat([df_train[[ORIGINAL_COLUMN, TRANSLATE_COLUMN]], 
                            df_train[[ORIGINAL_COLUMN, TRANSLATE_COLUMN_GPT]]. \
                            rename(columns={TRANSLATE_COLUMN_GPT: TRANSLATE_COLUMN})])
    
      df = df.append(df_train)
    
    else:
      df_train = pd.read_csv(
        filename_path,
        usecols=[ORIGINAL_COLUMN, TRANSLATE_COLUMN],
        dtype={ORIGINAL_COLUMN: str, TRANSLATE_COLUMN: str}
      )
    
      print(f'File name                          : {filename}')
      verifica_correttezza(df_train)

      df = df.append(df_train)

  return df.reset_index(drop=True)

In [34]:
# Caricamento dataset
df = union_df_train()

filename_path_out = os.path.abspath(os.path.join(DATA_PATH_OUT, file_out))

df.to_csv(filename_path_out, index=False, columns=[ORIGINAL_COLUMN, TRANSLATE_COLUMN], encoding='utf-8')

File name                          : inferno_v2.csv
Esempi nel Dataset                 : 1995
Frase più corta in Original        :  che hai?"
Frase più corta in Translate       :  Qui morì
Frase più lunga in Original        :   E già venìa su per le torbide onde un fracasso d’un suon, pien di spavento, per cui tremavano amendue le sponde,  non altrimenti fatto che d’un vento impetüoso per li avversi ardori, che fier la selva e sanz’alcun rattento  li rami schianta, abbatte e porta fori
Frase più lunga in Translate       : Mi disse: - Beatrice, autentica lode di Dio, perché non soccorri colui che ti amò al punto da elevarsi al di sopra della schiera volgare?Non senti l'angoscia del suo pianto? non vedi la morte che combatte sul gorgo tempestoso del peccato? -Al mondo non ci furono mai persone tanto rapide a perseguire il loro vantaggio o a fuggire il loro danno, quanto io, dopo aver udito quelle parole, venni quaggiù dal mio scanno celeste, affidandomi alle tue parole nobili che onorano

In [35]:
df

,Original,Translate
0,Lo Dio d’Amor con su’ arco mi trasse,Con il suo arco il Dio dell'Amore mi trafisse
1,"Perch’i’ guardava un Fior, che m’abellia,","mentre guardavo un fiore che mi piaceva,"
2,Lo quale avea piantato Cortesia,il quale aveva piantato Cortesia
3,"Nel Giardin di Piacier, e que’ vi trasse",nel giardino del Piacere. L'amore è venuto
4,"Sì tosto, c’a me parve ch’e’ volasse.","così in fretta che mi parve volare,"
...,...,...
90142,veder voleva come si convenne l’imago al cerc...,"Ich wollte herausfinden, wo man das Bild des M..."
90143,ma non eran da ciò le proprie penne,Meine Flügel waren nicht genug für einen solch...
90144,se non che la mia mente fu percossa da un ful...,"Plötzlich schlug mich ein Blitz in den Sinn, s..."
90145,A l’alta fantasia qui mancò possa,Meine Fantasie konnte die Höhe nicht halten.


In [14]:
verifica_correttezza(df)

Esempi nel Dataset                 : 90150
Frase più corta in Original        : 5
Frase più corta in Translate       : 5
Frase più lunga in Original        :   E già venìa su per le torbide onde un fracasso d’un suon, pien di spavento, per cui tremavano amendue le sponde,  non altrimenti fatto che d’un vento impetüoso per li avversi ardori, che fier la selva e sanz’alcun rattento  li rami schianta, abbatte e porta fori
Frase più lunga in Translate       : or to any other argument that is not of any relevance to the Court of Justice of the European Communities, or to any other argument that is not of any relevance to the Court of Justice of the European Communities, or to any other argument that is not of any relevance to the Court of Justice of the European Communities, or to any other argument that is not of any relevance to the Court of Justice of the European Communities, or to any other argument that is not of any relevance to the Court of Justice of the European Union, or to any o